# METRIC-BASED META-LEARNING using Matching Networks

In [ ]:
!pip install import_ipynb --quiet

In [ ]:
!pip install learn2learn --quiet

     |████████████████████████████████| 841 kB 3.8 MB/s 
     |████████████████████████████████| 2.9 MB 41.2 MB/s 
     |████████████████████████████████| 135 kB 48.8 MB/s 
     |████████████████████████████████| 96 kB 6.1 MB/s 
     |████████████████████████████████| 55 kB 3.9 MB/s 
     |████████████████████████████████| 1.4 MB 36.9 MB/s 
     |████████████████████████████████| 3.6 MB 25.8 MB/s 


In [ ]:
import requests

In [ ]:
r=requests.get("https://github.com/gmshroff/metaLearning2022/blob/main/code/utils.ipynb?raw=true")
with open('./utils.ipynb','wb') as f: f.write(r.content)

In [ ]:
r=requests.get("https://github.com/gmshroff/metaLearning2022/blob/main/code/l2lutils.ipynb?raw=true")
with open('./l2lutils.ipynb','wb') as f: f.write(r.content)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# %ls drive/MyDrive/'Colab Notebooks'/'Meta_Learning_hw2'
!cp drive/MyDrive/'Colab Notebooks'/'Meta_Learning_hw2'/models.ipynb .
!cp drive/MyDrive/'Colab Notebooks'/'Meta_Learning_hw2'/MAN.ipynb .


Mounted at /content/drive


In [ ]:
#For autoreload
# %load_ext autoreload
# %aimport MAN
# %autoreload 1

In [ ]:
import import_ipynb
import utils
import models
from MAN import MAN
from l2lutils import KShotLoader
from IPython import display
import torch
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
from IPython import display
from time import time



importing Jupyter notebook from utils.ipynb
importing Jupyter notebook from models.ipynb
importing Jupyter notebook from MAN.ipynb
importing Jupyter notebook from l2lutils.ipynb
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data Generation and Loading

In [ ]:
#Generate data - euclidean
meta_train_ds, meta_test_ds, full_loader = utils.euclideanDataset(n_samples=10000,n_features=20,n_classes=10,batch_size=32)

In [ ]:
#I added the code block
print(meta_train_ds.samples.shape)
res=0
for samples,labels in full_loader:
  # print(samples.shape)
  res += samples.shape[0]


print(res)

torch.Size([7500, 20])
7500


In [ ]:
# Define an MLP network. Note that input dimension has to be data dimension. For classification
# final dimension has to be number of classes; for regression one.
#torch.manual_seed(10)
net = models.MLP(dims=[20,32,32,10])

In [ ]:
# Train the network; note that network is trained in place so repeated calls further train it.
net,loss,accs=models.Train(net,full_loader,lr=1e-2,epochs=50,verbose=True)

Epoch   49 Loss: 3.71706e-02 Accuracy: 0.99215


In [ ]:
#Training accuracy.
models.accuracy(net,meta_train_ds.samples,meta_train_ds.labels,verbose=True)

7439.0 7500


0.9918666666666667

In [ ]:
# Test accuracy.
models.accuracy(net,meta_test_ds.samples,meta_test_ds.labels)

2405.0 2500


0.962

# Meta-Learning: Tasks

Generate a k-shot n-way loader using the meta-training dataset

In [ ]:
meta_train_kloader=KShotLoader(meta_train_ds,shots=5,ways=5)

Sample a task - each task has a k-shot n-way training set and a similar test set

In [ ]:
d_train,d_test=meta_train_kloader.get_task()

In [ ]:
# I added
print(d_test[1])
print(d_test[1].shape)

tensor([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 2, 2, 2, 2,
        2])


Let's try directly learning using the task training set albeit its small size: create a dataset and loader and train it with the earlier network and Train function.

In [ ]:
taskds = utils.MyDS(d_train[0],d_train[1])

In [ ]:
d_train_loader = torch.utils.data.DataLoader(dataset=taskds,batch_size=1,shuffle=True)

In [ ]:
net,loss,accs=models.Train(net,d_train_loader,lr=1e-1,epochs=10,verbose=False)

How does it do on the test set of the sampled task?

In [ ]:
models.accuracy(net,d_test[0],d_test[1])

# Matching Networks

In [ ]:
import learn2learn as l2l
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

Sampling a training task: Note that each of d_train and d_test is a tuple comprising of a training set, and labels.

In [ ]:
meta_train_kloader=KShotLoader(meta_train_ds,shots=5,ways=5)

In [ ]:
d_train,d_test=meta_train_kloader.get_task()

In [ ]:
lossfn = torch.nn.NLLLoss()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%ls drive/MyDrive/'Colab Notebooks'/Meta_Learning_hw2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
CNP.ipynb          l2lutils.ipynb  models.ipynb  nb3-CNP.ipynb  utils.ipynb
course_data.ipynb  MAN.ipynb       nb2.ipynb     nb3-MAN.ipynb


In [ ]:
!cp drive/MyDrive/'Colab Notebooks'/Meta_Learning_hw2/models.ipynb .
!cp drive/MyDrive/'Colab Notebooks'/Meta_Learning_hw2/MAN.ipynb .

In [ ]:
import models

This will be the class implementing Matching networks that you need to write for hw3

In [ ]:
from MAN import MAN

importing Jupyter notebook from MAN.ipynb


In [ ]:
X = torch.Tensor([[1,1,1],[-1,-1,-1],[1,2,3],[-1,-2,-3]])
y_tr = torch.LongTensor([0,1])
x_tr = X[[0,1],:]
d_tr = (x_tr,y_tr)

In [ ]:
man = MAN(dims=[3,8,8]) #n_classes=2

In [ ]:
man(X,d_tr)

NameError: ignored

# Putting it all together: Training a Matching Network
Now let's put all of the above in a loop - training Matching Network algorithm:

In [ ]:
final_time=0

In [ ]:
import import_ipynb
import utils
import models
from MAN import MAN
from l2lutils import KShotLoader
from IPython import display
import torch
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
from IPython import display
from time import time


In [ ]:
# Redifning accuracy function so that it takes h - dataset context - as input since net requires it.
def accuracy(Net,X_test,y_test,h,verbose=True):
    #Net.eval()
    m = X_test.shape[0]
    y_pred = Net(X_test,h)
    _, predicted = torch.max(y_pred, 1)
    correct = (predicted == y_test).float().sum().item()
    if verbose: print(correct,m)
    accuracy = correct/m
    #Net.train()
    return accuracy

In [ ]:
classes_train = [i for i in range(5)]
classes_test = [i+3 for i in range(5)]
classes_train, classes_test

([0, 1, 2, 3, 4], [3, 4, 5, 6, 7])

In [ ]:
# !cp drive/MyDrive/'Colab Notebooks'/'Meta_Learning_hw2'/MAN.ipynb .
# from MAN import MAN

In [ ]:
import learn2learn as l2l
import torch.optim as optim

#Generate data - euclidean
meta_train_ds, meta_test_ds, full_loader = utils.euclideanDataset(n_samples=10000,n_features=20,n_classes=10,batch_size=32)
shots,ways = 5,5
net = MAN(n_classes=ways,dims=[20,64,32],lr=1e-4)
lossfn = torch.nn.NLLLoss()
meta_train_kloader=KShotLoader(meta_train_ds,shots=shots,ways=ways,num_tasks=1000, classes=classes_train)
meta_test_kloader=KShotLoader(meta_test_ds,shots=shots,ways=ways, classes=classes_test)


In [ ]:
epoch=0
n_epochs=100
task_count=50
start_time=time()
while epoch<n_epochs:
    test_loss = 0.0
    test_acc = 0.0
    # Sample and train on a task
    for task in range(task_count):
        d_train,d_test=meta_train_kloader.get_task()
        rp = torch.randperm(d_train[1].shape[0]) # 0 to 24 randomly permuted d_train[1].shape=25
        d_train0=d_train[0][rp]
        d_train1=d_train[1][rp]
        x_tr = d_train0
        d_tr = x_tr 
        rp1 = torch.randperm(d_test[1].shape[0]) # 0 to 24 randomly permuted d_train[1].shape=25
        d_test0=d_test[0][rp1]
        d_test1=d_test[1][rp1]
        x_ts = d_test0
        d_ts = x_ts 
        test_preds = net(d_ts,(x_tr,d_train1))
        #train_preds = net(d_tr,h)
        # Accumulate losses over tasks - note train and test loss both included
        test_loss += lossfn(test_preds,d_test1)
        net.eval() #to activate the evaluation mode
        test_acc += accuracy(net,d_ts,d_test1,(x_tr,d_train1),verbose=False)
        net.train() #to activate the train mode
    
    #Update the network weights
    print('Epoch % 2d Loss: %2.5e Avg Acc: %2.5f'%(epoch,test_loss/task_count,test_acc/task_count))
    display.clear_output(wait=True)
    net.optimizer.zero_grad()
    test_loss.backward()
    net.optimizer.step()
    epoch+=1

time_taken=time()-start_time 


tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000],
       grad_fn=<SumBackward1>)


Now test the trained matching network and to tasks sampled from the meta_test_ds dataset:

In [ ]:
import numpy as np
final_time +=time_taken
print(final_time)
print(f"Time taken for {n_epochs} is: ", np.round(time_taken,3))


80.88686513900757
Time taken for 100 is:  80.887


In [ ]:
test_acc = 0.0
task_count = 50
adapt_steps = 1
net.train()

# Sample and train on a task
for task in range(task_count):
    d_train,d_test=meta_test_kloader.get_task()
    x_tr = d_train[0]
    y_tr_sh = torch.cat((torch.zeros(1,ways),torch.eye(ways)[d_train[1][1:]]))
    d_tr = x_tr #torch.cat((x_tr,y_tr_sh),1)
    x_ts = d_test[0]
    y_ts_sh = torch.zeros(x_ts.shape[0],ways)
    d_ts = x_ts #torch.cat((x_ts,y_ts_sh),1)
    test_preds = net(d_ts,(d_tr,d_train[1]))
    net.eval()
    test_acc += accuracy(net,d_ts,d_test[1],(d_tr,d_train[1]),verbose=False)
    # Done with a task
    net.train()
print('Avg Acc: %2.5f'%(test_acc/task_count))

Avg Acc: 0.68960
